# Exploring industry returns in the Fama-French data-set

The [49 Industry Portfolios](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_49_ind_port.html) data-set has a series of daily returns of 49 different industry types. Here, we look at some of their statistical properties.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
startDt <- as.Date("1990-01-01")

#value (market-cap) weighted
valueWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

#equal weighted
equalWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AEWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

In [3]:
lookbackDays <- 220 * 5

rrValueWtd <- rollapply(valueWtd, lookbackDays, Return.cumulative, by.column = F)
rrEqualWtd <- rollapply(equalWtd, lookbackDays, Return.cumulative, by.column = F)

rrValueWtd <- na.omit(rrValueWtd)
rrEqualWtd <- na.omit(rrEqualWtd)

names(rrValueWtd) <- names(valueWtd)
names(rrEqualWtd) <- names(equalWtd)

In [4]:
minValWtd <- apply(rrValueWtd, 2, min)
minValWtd <- data.frame(minValWtd)
print(minValWtd)

minEqlWtd <- t(apply(rrEqualWtd, 2, min))

maxValWtd <- apply(rrValueWtd, 2, max)
maxEqlWtd <- apply(rrEqualWtd, 2, max)

sdValWtd <- apply(rrValueWtd, 2, sd)
sdEqlWtd <- apply(rrEqualWtd, 2, sd)

statsValDf <- merge.data.frame(merge.data.frame(minValWtd, maxValWtd), sdValWtd)
statsEqlDf <- merge.data.frame(merge.data.frame(minEqlWtd, maxEqlWtd), sdEqlWtd)

print("MKT CAP WT")
print(statsValDf)

print("EQ WT")
print(statsEqlDf)

       minValWtd
AERO  -0.2532453
AGRIC -0.4074237
AUTOS -0.7431826
BANKS -0.7484332
BEER  -0.2183618
BLDMT -0.5767641
BOOKS -0.7936408
BOXES -0.4499764
BUSSV -0.4315746
CHEMS -0.3609480
CHIPS -0.7657270
CLTHS -0.3782547
CNSTR -0.5927093
COAL  -0.9631609
DRUGS -0.2876000
ELCEQ -0.3043374
FABPR -0.6453013
FIN   -0.6100470
FOOD  -0.1204453
FUN   -0.6996230
GOLD  -0.7739683
GUNS  -0.3086047
HARDW -0.7421280
HLTH  -0.3694879
HSHLD -0.2153368
INSUR -0.5480310
LABEQ -0.5442701
MACH  -0.3163298
MEALS -0.2264927
MEDEQ -0.2991468
MINES -0.5143283
OIL   -0.3595542
OTHER -0.5805491
PAPER -0.5082174
PERSV -0.3867057
RLEST -0.7545607
RTAIL -0.2353749
RUBBR -0.4900953
SHIPS -0.3139155
SMOKE -0.2988185
SODA  -0.4583180
SOFTW -0.6970507
STEEL -0.6504543
TELCM -0.6277940
TOYS  -0.5645464
TRANS -0.3104545
TXTLS -0.7562049
UTIL  -0.1921551
WHLSL -0.2634272
[1] "MKT CAP WT"
[1] y         minValWtd
<0 rows> (or 0-length row.names)
[1] "EQ WT"
 [1] y     AERO  AGRIC AUTOS BANKS BEER  BLDMT BOOKS BOXES BUSSV

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)